# Character-level recurrent sequence-to-sequence model

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


## Setup


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from numpy.random import seed
seed(1)

import pickle
from tqdm import tqdm

## Download the data


In [2]:
# TUTORIAL DATA
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip


['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

In [3]:
# Indicate dataframes to import.
list_dfs = ['pickled_conala_mined_df', 'pickled_conala_train_df', 'pickled_conala_test_df',
           'conala_train_bag_df', 'conala_mined_bag_df', 'combined_bag_df']

In [4]:
%time
# Load all data in list_dfs
data = {}
for df in list_dfs:
    dbfile = open(df, 'rb')      
    contents = pickle.load(dbfile)
    data[df] = contents
    dbfile.close()

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.81 µs


In [5]:
data.keys()

dict_keys(['pickled_conala_mined_df', 'pickled_conala_train_df', 'pickled_conala_test_df', 'conala_train_bag_df', 'conala_mined_bag_df', 'combined_bag_df'])

In [6]:
df = data["pickled_conala_train_df"]

In [7]:
df.head()

,intent,rewritten_intent,snippet,question_id
0,How to convert a list of multiple integers int...,Concatenate elements of a list 'x' of multiple...,"sum(d * 10 ** i for i, d in enumerate(x[::-1]))",41067960
1,How to convert a list of multiple integers int...,convert a list of integers into a single integer,"r = int(''.join(map(str, x)))",41067960
2,how to convert a datetime string back to datet...,convert a DateTime string back to a DateTime o...,datetime.strptime('2010-11-13 10:33:54.227806'...,4170655
3,Averaging the values in a dictionary based on ...,get the average of a list values for each key ...,"[(i, sum(j) / len(j)) for i, j in list(d.items...",29565452
4,zip lists in python,"zip two lists `[1, 2]` and `[3, 4]` into a lis...","zip([1, 2], [3, 4])",13704860


In [8]:
# There's some Nans in rewritten intent. Drop them.
drops = df[df["rewritten_intent"].isna()].index
df = df.drop(drops)

In [9]:
# prep takes lists of text.
input_list=list(df["snippet"])
target_list=list(df["rewritten_intent"])

## Configuration


In [10]:
batch_size = 128  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

## Prepare the data


In [11]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for input_text, target_text in zip(input_list, target_list):
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 2300
Number of unique input tokens: 98
Number of unique output tokens: 108
Max sequence length for inputs: 232
Max sequence length for outputs: 185


In [13]:
target_texts

["\tConcatenate elements of a list 'x' of multiple integers to a single integer\n",
 '\tconvert a list of integers into a single integer\n',
 "\tconvert a DateTime string back to a DateTime object of format '%Y-%m-%d %H:%M:%S.%f'\n",
 '\tget the average of a list values for each key in dictionary `d`)\n',
 '\tzip two lists `[1, 2]` and `[3, 4]` into a list of two tuples containing elements at the same index in each list\n',
 "\tprepend string 'hello' to all items in list 'a'\n",
 '\tregex for repeating words in a string `s`\n',
 '\tnormalize a pandas dataframe `df` by row\n',
 '\tswap values in a tuple/list inside a list `mylist`\n',
 '\tSwap values in a tuple/list in list `mylist`\n',
 '\tfind all occurrences of the pattern \'\\\\[[^\\\\]]*\\\\]|\\\\([^\\\\)]*\\\\)|"[^"]*"|\\\\S+\' within `strs`\n',
 '\tgenerate the combinations of 3 from a set `{1, 2, 3, 4}`\n',
 '\tadd multiple columns `hour`, `weekday`, `weeknum` to pandas data frame `df` from lambda function `lambdafunc`\n',
 "\tB

## Build the model


In [14]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Train the model


In [106]:
import os

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

if not os.path.exists('models/s2s'):
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
    )
    # Save model
    model.save('models/s2s')

else:
    print("No need to train, model can be loaded." \
          " Move on to next step...")
    

No need to train, model can be loaded. Move on to next step...


## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [102]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("models/s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


You can now generate decoded sentences as such:


In [45]:
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: sum(d * 10 ** i for i, d in enumerate(x[::-1]))
Decoded sentence: cont ale ing of alesting ` on  on  on  on  on  on  on  on  ing `

-
Input sentence: r = int(''.join(map(str, x)))
Decoded sentence: cont ale ing of alesting ` on  on  on  on  on  on  on  on  ing `

-
Input sentence: datetime.strptime('2010-11-13 10:33:54.227806', '%Y-%m-%d %H:%M:%S.%f')
Decoded sentence: cont ale ing of alesting ` on  on  on  on  on  on  on  on  ing `

-
Input sentence: [(i, sum(j) / len(j)) for i, j in list(d.items())]
Decoded sentence: cont ale ing of alesting ` on  on  on  on  on  on  on  on  ing `

-
Input sentence: zip([1, 2], [3, 4])
Decoded sentence: cont ale ing of alesting ` on  on  on  on  on  on  on  on  ing `



# Testing with Prob Char Selection

Previously, used argmax

In [25]:
# Testing output cell
input_seq = encoder_input_data[0 : 0 + 1]
states_value = encoder_model.predict(input_seq)

In [30]:
input_seq

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]]], dtype=float32)

In [101]:
states_value[0].shape
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, target_token_index["\t"]] = 1.0
stop_condition = False
decoded_sentence = ""

# output_tokens is the prob list
predict_proba, h, c = decoder_model.predict([target_seq] + states_value)

sampled_char = np.random.choice(target_characters, 1, p = predict_proba.flatten())[0]
new = predict_proba.flatten()
new.sort()
new[-1::-1]
#print(predict_proba[-1],predict_proba[-2])

array([1.87475666e-01, 1.36296734e-01, 1.02006570e-01, 8.72212723e-02,
       4.59922403e-02, 3.88737358e-02, 3.20781991e-02, 3.16622220e-02,
       3.14001814e-02, 2.93414313e-02, 2.70991214e-02, 2.69555338e-02,
       2.22501475e-02, 2.03606095e-02, 1.79793164e-02, 1.74739361e-02,
       1.52271641e-02, 1.48214865e-02, 1.28578003e-02, 8.88201687e-03,
       8.35316442e-03, 8.27643275e-03, 7.33574759e-03, 6.20183023e-03,
       6.13153400e-03, 5.80775598e-03, 4.58229380e-03, 4.48314939e-03,
       3.72113450e-03, 3.69795295e-03, 3.46529414e-03, 3.30777117e-03,
       3.14853177e-03, 3.00594186e-03, 2.26853346e-03, 2.24035163e-03,
       2.00786488e-03, 1.93919579e-03, 1.72736298e-03, 1.37315737e-03,
       1.30308256e-03, 1.25243282e-03, 1.06828671e-03, 9.42941289e-04,
       8.21999507e-04, 5.72410645e-04, 5.30742225e-04, 4.89771715e-04,
       4.46277903e-04, 3.88717803e-04, 1.95246641e-04, 1.91258077e-04,
       1.60438882e-04, 1.40379096e-04, 1.36431700e-04, 1.16656222e-04,
      

In [70]:
# Use decode sequence with random choice on predicted probs. 
def decode_sequence_prob(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        predict_proba, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_char = np.random.choice(target_characters, 1, p = predict_proba.flatten())[0]
        sampled_token_index = target_characters.index(sampled_char)
        
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [72]:
# Testing probability selection
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence_prob(input_seq) # change decode func
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: sum(d * 10 ** i for i, d in enumerate(x[::-1]))
Decoded sentence: eptatet `xlike `aontronevateipaamnm'teun 'rotaminchasw oin inetsmatnifranminrsk ,   i   i                                                                                                 
-
Input sentence: r = int(''.join(map(str, x)))
Decoded sentence: retsotrodfromy `wot eaistgengdint3it

-
Input sentence: datetime.strptime('2010-11-13 10:33:54.227806', '%Y-%m-%d %H:%M:%S.%f')
Decoded sentence: reoratharan ilmliqtonectoems stiTat an wenlemesring ihonuemsnncouesu

-
Input sentence: [(i, sum(j) / len(j)) for i, j in list(d.items())]
Decoded sentence: Tcray telssling '0 isl fmestrong

-
Input sentence: zip([1, 2], [3, 4])
Decoded sentence: wetroie`sithexn heme ont orlecing prrthtinfo, 'ici `ulanuse,k` "remnsg p umsrainu`` nof  acot unpy``ain 'mirh'd`

-
Input sentence: ['hello{0}'.format(i) for i in a]
Decoded sentence: smitt alelpss16ofhile 2r32 ihe ding ockeerilg iltge` dw`Apnadcoptiitkafs lgsbodese nod